Install Required Packages

In [1]:
!pip install pathway
!pip install pandas


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.8 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of boto3 to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.9/68.9 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 39.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 72.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.4/318.4 kB 24

Upload and Preprocess Base Parking Data

In [1]:
import pandas as pd

# Load parking lot data
parking_df = pd.read_csv("/content/dataset.csv")

# Combine date and time into a timestamp
parking_df['TimestampStr'] = parking_df['LastUpdatedDate'] + ' ' + parking_df['LastUpdatedTime']

# Save updated dataset
parking_df.to_csv("dataset_with_timestamp.csv", index=False)


Create Synthetic External Events Data

In [2]:
# Create dummy external events data
external_data = pd.DataFrame({
    'Timestamp': parking_df['TimestampStr'].unique()[:10],
    'EventType': ['concert', 'sports', 'none', 'none', 'festival', 'none', 'none', 'concert', 'none', 'sports'],
    'Weather': ['sunny', 'rainy', 'sunny', 'cloudy', 'sunny', 'rainy', 'cloudy', 'sunny', 'sunny', 'cloudy']
})

external_data.to_csv("external_events_weather.csv", index=False)


Define Schemas in Pathway

In [3]:
import pathway as pw

class ParkingSchema(pw.Schema):
    ID: str
    SystemCodeNumber: str
    Capacity: int
    Latitude: float
    Longitude: float
    Occupancy: int
    VehicleType: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int
    LastUpdatedDate: str
    LastUpdatedTime: str
    TimestampStr: str

class ExternalSchema(pw.Schema):
    Timestamp: str
    EventType: str
    Weather: str


Load Streaming Tables

In [4]:
# Streaming parking data
parking = pw.io.csv.read(
    "dataset_with_timestamp.csv",
    schema=ParkingSchema,
    mode="streaming",
    autocommit_duration_ms=1000
)

# Streaming external events/weather
external = pw.io.csv.read(
    "external_events_weather.csv",
    schema=ExternalSchema,
    mode="streaming",
    autocommit_duration_ms=1000
)


Define UDFs for Demand and Pricing

In [5]:
@pw.udf
def compute_adjusted_demand(occupancy, capacity, queue, special_day, event, weather):
    if capacity == 0:
        return 0.0

    # Base demand from occupancy and queue
    demand = (occupancy / capacity) + 0.5 * queue + 1.2 * special_day

    # Event impact
    event_impact = {'concert': 2.0, 'sports': 1.5, 'festival': 1.8, 'none': 0.0}
    demand += event_impact.get(event, 0.0)

    # Weather impact
    weather_impact = {'rainy': -1.0, 'cloudy': -0.5, 'sunny': 0.5}
    demand += weather_impact.get(weather, 0.0)

    return max(demand, 0.0)

@pw.udf
def price_from_demand(demand):
    base_price = 10
    lambda_ = 1.5
    return min(max(base_price * (1 + lambda_ * demand), base_price * 0.5), base_price * 2.5)


Join Parking and External Tables

In [6]:
# Join on timestamp
joined = parking.join(external, parking.TimestampStr == external.Timestamp)

# Compute demand and price
result = joined.select(
    ID=joined.ID,
    Timestamp=joined.TimestampStr,
    Demand=compute_adjusted_demand(
        joined.Occupancy,
        joined.Capacity,
        joined.QueueLength,
        joined.IsSpecialDay,
        joined.EventType,
        joined.Weather
    )
)

# Compute final price
result += result.select(Price=price_from_demand(result.Demand))


Export the Result as Streaming Output

In [ ]:
# Write to JSON Lines file
pw.io.jsonlines.write(result, "model3_output.jsonl")

# Execute Pathway pipeline
pw.run()


Output()

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(
